In [275]:
import nltk
import string
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.metrics import classification_report
import os
import urllib.request   
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM,Flatten
import numpy as np




In [276]:
if not os.path.exists('data'):
    os.mkdir('data')

urllib.request.urlretrieve('https://www.gutenberg.org/files/2591/2591-0.txt', 'data/Jacob and Wilhelm/grimms_fairy_tales.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/345/pg345.txt', 'data/Bram Stoker/dracula.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/84/pg84.txt', 'data/Mary Shelley/frankenstein.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/files/2701/2701-0.txt', 'data/Herman Melville/moby_dick.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/files/74/74-0.txt', 'data/Mark Twain/tom_sawyer.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/38189/pg38189.txt', 'data/Ambrose Blacklock/A_TREATISE_ON_SHEEP.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/2166/pg2166.txt', 'data/H.Rider Haggard/King_Solomons_Mines.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/21728/pg21728.txt', 'data/R.M.Ballantyne/THE_DOG_CRUSOE_AND_HIS_MASTER.txt')
urllib.request.urlretrieve('https://www.gutenberg.org/files/95/95-0.txt', 'data/Anthony Hope/THE_PRISONER_OF_ZENDA.txt')
urllib.request.urlretrieve('https://www.gutenberg.org/files/1257/1257-0.txt', 'data/Alexandre Dumas/THE_THREE_MUSKETEERS.txt')


('data/Anthony Hope/THE_PRISONER_OF_ZENDA.txt',
 <http.client.HTTPMessage at 0x2a83878f470>)

In [277]:
path = './data'
token_dict = {}
dfs=[]



for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        fdir=os.path.basename(dirpath)
        print ("fname=", fname)
        dt=pd.read_fwf(fname, sep=".", header=None)
        dt['label']=fdir
        dfs.append(dt)

            
big_frame = pd.concat(dfs, ignore_index=True)      

fname= ./data\Alexandre Dumas\THE_THREE_MUSKETEERS.txt
fname= ./data\Ambrose Blacklock\A_TREATISE_ON_SHEEP.txt
fname= ./data\Anthony Hope\THE_PRISONER_OF_ZENDA.txt
fname= ./data\Bram Stoker\dracula.txt
fname= ./data\H.Rider Haggard\King_Solomons_Mines.txt
fname= ./data\Herman Melville\moby_dick.txt
fname= ./data\Jacob and Wilhelm\grimms_fairy_tales.txt
fname= ./data\Mark Twain\tom_sawyer.txt
fname= ./data\Mary Shelley\frankenstein.txt
fname= ./data\R.M.Ballantyne\THE_DOG_CRUSOE_AND_HIS_MASTER.txt


C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [279]:
big_frame.head()

,0,label,1
0,ï»¿ THE THREE MUSKETEERS,Alexandre Dumas,NaN
1,This ebook is for the use of anyone anywhere i...,Alexandre Dumas,NaN
2,most other parts of the world at no cost and w...,Alexandre Dumas,NaN
3,"whatsoever. You may copy it, give it away or r...",Alexandre Dumas,NaN
4,of the Project Gutenberg License included with...,Alexandre Dumas,NaN


In [280]:
big_frame["label"].value_counts()

Alexandre Dumas      22602
Herman Melville      19220
Bram Stoker          13414
Jacob and Wilhelm     9571
R.M.Ballantyne        7830
H.Rider Haggard       7451
Mark Twain            6927
Ambrose Blacklock     6577
Anthony Hope          5206
Mary Shelley          1300
Name: label, dtype: int64

In [281]:
big_frame.dtypes
big_frame[0] = big_frame[0].replace('"', ' ', regex=True)
big_frame[0] = big_frame[0].replace('!', ' ', regex=True)
big_frame[0] = big_frame[0].replace('-', ' ', regex=True)
big_frame[0] = big_frame[0].replace(',', ' ', regex=True)
big_frame[0] = big_frame[0].replace('_', ' ', regex=True)
big_frame[0] = big_frame[0].replace(';', ' ', regex=True)



In [282]:
df_x=big_frame[0].values.astype('U')
df_y=big_frame["label"]


## 1/ Naive Bayes classifier 

In [325]:

cv = TfidfVectorizer(min_df=1,ngram_range =(1,2),stop_words='english')
x_trainc=cv.fit_transform(df_x)
x_train, x_test, y_train, y_test = train_test_split(x_trainc, df_y, test_size=0.1, random_state=4)
mnb = MultinomialNB(alpha=0.1)
clf=mnb.fit(x_train,y_train)


In [326]:
cv.get_feature_names()


['00',
 '00 lˆð',
 '000',
 '000 000',
 '000 26',
 '000 500',
 '000 annually',
 '000 annum',
 '000 cloths',
 '000 comes',
 '000 demanded',
 '000 dollars',
 '000 edward',
 '000 elephants',
 '000 excessive',
 '000 fast',
 '000 holland',
 '000 lastly',
 '000 lbs',
 '000 men',
 '000 miles',
 '000 north',
 '000 particularly',
 '000 people',
 '000 persons',
 '000 pieces',
 '000 pounds',
 '000 seized',
 '000 shall',
 '000 sheep',
 '000 sterling',
 '000 swedes',
 '000 total',
 '000 value',
 '000 wages',
 '000 west',
 '000 wool',
 '000 woollens',
 '000 worth',
 '000 year',
 '007',
 '007 198',
 '01',
 '01 1998',
 '013',
 '013 12',
 '014',
 '014 quantity',
 '01ó',
 '02',
 '02 òî',
 '020',
 '020 lbs',
 '023',
 '023 lbs',
 '025',
 '025 worth',
 '02b',
 '02å',
 '02å iá¾q',
 '04',
 '04 ìèµ',
 '042',
 '042 yards',
 '045',
 '045 240',
 '04ïø',
 '04ïø pö',
 '059',
 '059 lbs',
 '06',
 '06 rœ',
 '061',
 '061 worsted',
 '06ëbc7',
 '06ëbc7 ùœß',
 '072',
 '072 yards',
 '073',
 '09',
 '09 fù',
 '09²ðjeþfùé',
 

In [327]:
predicted = clf.predict(x_test)

In [328]:
print('%.1f%%' % (np.mean(predicted == y_test) * 100))

68.2%


In [329]:
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.64      0.87      0.74      2286
Ambrose Blacklock       0.73      0.64      0.68       637
     Anthony Hope       0.82      0.35      0.49       506
      Bram Stoker       0.60      0.68      0.64      1364
  H.Rider Haggard       0.76      0.44      0.56       785
  Herman Melville       0.68      0.77      0.72      1882
Jacob and Wilhelm       0.73      0.72      0.73       981
       Mark Twain       0.80      0.45      0.58       666
     Mary Shelley       1.00      0.61      0.76       139
   R.M.Ballantyne       0.73      0.57      0.64       764

      avg / total       0.70      0.68      0.67     10010



## 2/ Stochastic Gradient Descent classifier

In [330]:
svm=SGDClassifier()
svm_classifier=svm.fit(x_train,y_train)

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [331]:
svm_predicted = svm_classifier.predict(x_test)
print('%.1f%%' % (np.mean(svm_predicted == y_test) * 100))

65.3%


In [332]:
print(classification_report(y_test, svm_predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.64      0.84      0.73      2286
Ambrose Blacklock       0.64      0.68      0.66       637
     Anthony Hope       0.74      0.35      0.47       506
      Bram Stoker       0.64      0.61      0.62      1364
  H.Rider Haggard       0.72      0.40      0.51       785
  Herman Melville       0.62      0.76      0.68      1882
Jacob and Wilhelm       0.70      0.69      0.70       981
       Mark Twain       0.71      0.44      0.54       666
     Mary Shelley       0.90      0.33      0.48       139
   R.M.Ballantyne       0.68      0.55      0.61       764

      avg / total       0.66      0.65      0.64     10010

